![](Picture1.png)  

# <span style="color:navy"> <i> Seminar 03</i></span> : <span style="color:green"> <b>Video Coding</b> </span>

## <span style="color:navy"> <b>TASK:</b> Compression of a Video using a Color Transform and Chroma Downsampling </span>
### <span style="color:blue"> **TASK 1:** Design an Encoder: </span>
- Modify given Encoder function `def encoder():` use video file `videorec.mp4` (it is given in your assignment directory), and implement the Encoder with a **color transform** of your choice. For example you can use the color transformation *YCrCB* or *YIQ*.
- Next, include **chroma downsampling** of your choice.
  - You can try different *color-transforms* and *downsampling rates* to minimize the encoded file size and maximize the decoded visual quality.  

<span style="color:red">**NOTE:**</span> Represent these components as **int8** or **uint8**, and store them in a pickle file `encoded.pickle` as your encoded file.

### <span style="color:blue"> **TASK 2:** Design a Decoder: </span>
- Modify the corresponding given Decoder function `def decoder():`, which must take the produced **pickle file** `encoded.pickle` as new input for your decoder and writes this  output decoded video in a file named `decoded.mp4`.  
- Execute your Encoder and Decoder functions and watch the resulting videos for corss checking in order to decide if the produced video looks reasonable and plausible.

### <span style="color:green"> **Grade Points:**  </span>  
The grade points for your results will be calculated from a weighted average of the compression ratio (i.e. *the size of the final decoded video file divided by the size of the compressed encoded pickle file*.) and the reconstructed video quality as measrued be `ffmpeg ssim` (i.e. with values between $0$ and $1$ for best. See also lectures for more information about SSIM: How it works):  

$$logcompression=np.log(\frac{decodedsize}{compressedsize})$$  

$$performance=logcompression+5*ssimval$$  

1. If the performance $>0.3$, you will get total $4$ points.
2. If the performance $>0.5$, you will additional $2$ points. (Total $6$ points)
3. If the performance $>0.7$, you will another additional $2$ points. (Total $8$ points)

In [1]:
import os
import sys
import cv2
import numpy as np
from IPython.display import Video
import pickle5 as pickle
import scipy.signal
import scipy.signal as sp
import matplotlib.pyplot as plt

### <span style="color:green"> **Input Video**</span>
Watch the original input video

In [2]:
from moviepy.editor import *
clip = VideoFileClip("videorec.mp4")
clip.subclip(0,3)
clip.ipython_display()

ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No su

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


## <span style="color:red"> START Your Solution </span>
### <span style="color:navy"> **Taks I:** Design Encoder </span>
Complete the missing part in the `def encoder():`

In [25]:
def encoder():
    # Reading the video file
    cap = cv2.VideoCapture('videorec.mp4')
    # Open a new blank file to save resultant envode
    g = open('encoded.pickle', 'wb')
    # Read Video frame by frame
    # Create a list to store encoded frames
    encoded_frames = []
    while(cap.isOpened()):
        ret, frame0 = cap.read()
        if ret==0:
            break
        
        # YOUR CODE HERE
        #raise NotImplementedError()
        # Define color transform and downsampling parameters
        color_transform = cv2.COLOR_BGR2YCrCb  # Choose the color transformation you prefer
        downsampling_ratio = 0.5  # Choose the downsampling ratio you prefer
        
        # apply color tranform
        frame_transformed = cv2.cvtColor(frame0, color_transform)

        # Split channel
        y, cr, cb = cv2.split(frame_transformed)

        # Perform downsampling on chroma channels (Cr and Cb)
        y_downsampled = cv2.resize(y, None, fx=downsampling_ratio, fy=downsampling_ratio)
        cr_downsampled = cv2.resize(cr, None, fx=downsampling_ratio, fy=downsampling_ratio)
        cb_downsampled = cv2.resize(cb, None, fx=downsampling_ratio, fy=downsampling_ratio)
        
        # Convert frame to int8 or uint8 representation
        y_downsampled_encoded = y_downsampled.astype(np.uint8)  
        cr_downsampled_encoded = cr_downsampled.astype(np.uint8) 
        cb_downsampled_encoded = cb_downsampled.astype(np.uint8) 

        # Merge channels back
        frame_encoded = cv2.merge([y_downsampled, cr_downsampled, cb_downsampled])

        # Store the encoded frame
        encoded_frames.append(frame_encoded)

    # Store the encoded frames in a pickle file
    pickle.dump(encoded_frames, g)

        #
        ### End Code before this line
    cap.release()
    return

### <span style="color:navy"> **Taks II:** Design Decoder </span>
Complete the missing part in the `def decoder():`
- Once you have save the Encoded video in `encoded.pickle`, now load it and apply a Decoder

In [26]:
def decoder():
    # Read your saved resultant video 'encoded.pickle'
    g = open('encoded.pickle', 'rb')
    # find frame dimentions
    height , width = (480, 640)
    # Define the codec and create VideoWriter object: In this case it is MP4
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    # Open empty file where to save decoded video output
    out = cv2.VideoWriter('decoded.mp4',fourcc, 20.0, (width,height))
    ctr = 0
    # Read video frame by frame and apply decoder
    #while(True):
        # Load next frame from file and "de-pickle" it. Convert from a string back to colortransform or tensor:
    #    ctr+=1
        # Load an encoded frame in Y:
    #    try:
    #        Y=pickle.load(g)
    #    except (EOFError):
    #        break
    #    DCr=pickle.load(g)
    #    DCb=pickle.load(g)
        
        # YOUR CODE HERE
    encoded_frames = pickle.load(g)

    #encoded_frames = pickle.load(g)

    for frames in encoded_frames:
        y_downsampled, cr_downsampled, cb_downsampled = cv2.split(frames)

            #upsampling 
        y_upsampled = cv2.resize(y_downsampled, (width,height))
        cr_upsampled = cv2.resize(cr_downsampled, (width,height))
        cb_upsampled = cv2.resize(cb_downsampled, (width,height))

            # Merge the channels back
        frame_decoded = cv2.merge([y_upsampled, cr_upsampled, cb_upsampled])

            # Convert the frame to BGR color space
        framedec = cv2.cvtColor(frame_decoded, cv2.COLOR_YCrCb2BGR)

            # Write the decoded frame to the output video file

            #raise NotImplementedError()
            #
            ### End Code before this line
        out.write(framedec)
    out.release()
    return

### <span style="color:navy"> Verify Encoder and Decoder </span>
- Once you have saved the Decoded video in `decoded.mp4`, now let us execute the `encoder()` and `decoder()` functions to see the resultant decoded video  
Play final video **decoded.mp4**, it should produce a reasoable results compared to original video

In [27]:
print("Encoder is Running ... ")
encoder()
print("Decoder is Running ... ")
decoder()
print("Completed executing Encoder and Decoder")
#Video("decoded.mp4", height=400, embed=True)
from moviepy.editor import *
clip = VideoFileClip("decoded.mp4")
clip.subclip(0,3)
clip.ipython_display()

Encoder is Running ... 
Decoder is Running ... 
Completed executing Encoder and Decoder
Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


### <span style="color:navy"> Estimate Correctness of Results</span>
The Verification cell will pass your results if **NO errors** are raised by `assert` function and fail otherwise!

In [28]:
# Let us run your encoder and decoder
print("Wait ... ")
encoder()
decoder()
print("Below are the statistics of your resultant decoded video")
print("")
compsize = os.path.getsize('encoded.pickle')
decsize = os.path.getsize('decoded.mp4')
logcompression = np.log(decsize/compsize)
print("Log Compression ratio  =  ", logcompression)
old_stdout = sys.stdout
sys.stdout = open('out.txt', 'w')
!ffmpeg -i videorec.mp4 -i decoded.mp4 -filter_complex "ssim" -f null /dev/null 
os.system("grep -oP '(?<=All:)[0-9.]+' out.txt > ssim.txt")
ssimval=np.loadtxt('ssim.txt')
sys.stdout = old_stdout
performance = logcompression + 5*ssimval
print("Your SSIM Value =  ", ssimval)
print("Your performance value is =  ", performance)
print("")
print("Compare the above statistics of your results with the grade points criteria in Instruction section")
print("You can also improve the performance value by try again to design ENCODER and DECODER functions above in order to get maximum points")

Wait ... 
Below are the statistics of your resultant decoded video

Log Compression ratio  =   -4.090550968785775
Your SSIM Value =   0.983824
Your performance value is =   0.8285690312142258

Compare the above statistics of your results with the grade points criteria in Instruction section
You can also improve the performance value by try again to design ENCODER and DECODER functions above in order to get maximum points


## <span style="color:red"> Evaluations of Resuls </span>
#### <span style="color:blue"> **NOTE:** The results will be evaluated and displayed in the feedback, after submission of the assignment

In [7]:
# First Test Results

In [8]:
# Second Test Results

In [9]:
# Third Test Results

## <span style="color:navy">TASK Completed: </span> <span style="color:green"> **Good Luck** </span>

### <span style="color:red"> STOP Here !!!! </span>